In [1]:
cd PerrosGatos

C:\Users\el_he\RNAs\T4 - Dogs & Cats\PerrosGatos


In [2]:
!git clone https://dagshub.com/jorgevc/Gatos-y-Perros.git

Cloning into 'Gatos-y-Perros'...


In [3]:
%cd Gatos-y-Perros

C:\Users\el_he\RNAs\T4 - Dogs & Cats\PerrosGatos\Gatos-y-Perros


In [5]:
pip install dvc>=2.8.1 --quiet

Note: you may need to restart the kernel to use updated packages.


In [1]:
!pip install dvc-s3 --quiet

In [2]:
!dvc remote add origin https://dagshub.com/jorgevc/Gatos-y-Perros.dvc -f
!more .dvc/config

ERROR: configuration error - config file error: Not inside a DVC repo


Par metro no v lido: /config


In [3]:
import os
import dagshub
import mlflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Input
from tensorflow.keras.optimizers import RMSprop, SGD, Adam
from mlflow import start_run, end_run, log_metric
from tensorflow.keras import regularizers
from keras.callbacks import ModelCheckpoint, EarlyStopping
from getpass import getpass
from IPython.core.magics import history
from mlflow.tracking.fluent import log_param
from tensorflow.python.platform.tf_logging import log
from mlflow.keras import log_model

C:\Users\el_he\anaconda3\lib\site-packages\pydantic\_internal\_fields.py:128: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
C:\Users\el_he\anaconda3\lib\site-packages\pydantic\_internal\_config.py:317: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


In [5]:
!pip3 install dagshub --upgrade 

In [6]:
dagshub.init("PerrosGatos", "RZeusHerMun", mlflow=True)

Repository initialized!

In [7]:
username = "RZeusHerMun"
reponame = "PerrosGatos"
repowner = "RZeusHerMun"
eta = 0.001

In [8]:
os.environ['MLFLOW_TRACKING_USERNAME'] = username
os.environ['MLFLOW_TRACKING_PASSSWORD'] = getpass('Password de Dagshub')


Password de Dagshub········


In [9]:
mlflow.tensorflow.autolog()

2023/10/02 23:47:53 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of tensorflow. If you encounter errors during autologging, try upgrading / downgrading tensorflow to a supported version, or try upgrading MLflow.


In [10]:
model = Sequential()
model.add(Input(shape=(28,28)))
model.add(Flatten()) 
model.add(Dense(512, activation='sigmoid'))
model.add(Dense(512, activation='relu', kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4))) #Regularizacion L1L2
model.add(Dropout(0.2)) #Fraccion de enlaces a eliminar
model.add(Dense(200)) #Capa lineal , transformacion lineal sin funcion de activacion
model.add(Activation('tanh')) #Se puede agregar despues la funcion de activacion
model.add(Dense(400, activation='selu', kernel_regularizer=regularizers.L1(0.01) )) #Regularizacion L1
model.add(Dense(200, activation='elu', kernel_regularizer=regularizers.L2(l2=1e-4)) ) #Regularizacion L2
model.add(Dense(50,activation='exponential'))
model.add(Dense(10, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 512)               401920    
                                                                 
 dense_1 (Dense)             (None, 512)               262656    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_2 (Dense)             (None, 200)               102600    
                                                                 
 activation (Activation)     (None, 200)               0         
                                                                 
 dense_3 (Dense)             (None, 400)               8

In [11]:
earlystop = EarlyStopping(monitor='val_loss',mode='min',restore_best_weights=True, patience=10,verbose=1)
filepath = "primer_modelo"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [12]:
dataset=mnist.load_data()
(x_train, y_train), (x_test, y_test) = dataset

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255  
x_test /= 255

num_classes=10
y_trainc = keras.utils.to_categorical(y_train, num_classes)
y_testc = keras.utils.to_categorical(y_test, num_classes)

In [13]:
mlflow.set_experiment(experiment_name="Experiment 2")

2023/10/02 23:48:30 INFO mlflow.tracking.fluent: Experiment with name 'Experiment 2' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/2f0522c3bd584d8d864a0a6449a050cf', creation_time=1696312110284, experiment_id='0', last_update_time=1696312110284, lifecycle_stage='active', name='Experiment 2', tags={}>

In [14]:
with mlflow.start_run(nested=True):
  mlflow.log_param("Tasa_aprendizaje", eta)

  model.compile(loss="categorical_crossentropy",optimizer=Adam(learning_rate=eta),metrics=['accuracy'])
  historial = model.fit(x_train, y_trainc,
                      batch_size=128,
                      epochs=5,
                      verbose=1,
                      validation_data=(x_test, y_testc),
                      callbacks=[earlystop, checkpoint])
  """mlflow.log_metric("loss", historial.history["loss"][-1])
  mlflow.log_metric("loss_validation", historial.history["val_loss"][-1])
  mlflow.log_metric("accuracy", historial.history["accuracy"][-1])
  mlflow.log_metric("accuracy_validation", historial.history["val_accuracy"][-1])"""
  loaded_model = tf.keras.models.load_model("primer_modelo")
  log_model(loaded_model, "mejores_modelos")
mlflow.end_run()

Epoch 1/5
468/469 [============================>.] - ETA: 0s - loss: 5.7785 - accuracy: 0.7939
Epoch 1: val_loss improved from inf to 0.72519, saving model to primer_modelo
INFO:tensorflow:Assets written to: primer_modelo\assets


INFO:tensorflow:Assets written to: primer_modelo\assets


469/469 [==============================] - 16s 31ms/step - loss: 5.7702 - accuracy: 0.7941 - val_loss: 0.7252 - val_accuracy: 0.9013
Epoch 2/5
469/469 [==============================] - ETA: 0s - loss: 0.5792 - accuracy: 0.9244
Epoch 2: val_loss improved from 0.72519 to 0.49847, saving model to primer_modelo
INFO:tensorflow:Assets written to: primer_modelo\assets


INFO:tensorflow:Assets written to: primer_modelo\assets


469/469 [==============================] - 21s 44ms/step - loss: 0.5792 - accuracy: 0.9244 - val_loss: 0.4985 - val_accuracy: 0.9435
Epoch 3/5
469/469 [==============================] - ETA: 0s - loss: 0.4444 - accuracy: 0.9495
Epoch 3: val_loss improved from 0.49847 to 0.38704, saving model to primer_modelo
INFO:tensorflow:Assets written to: primer_modelo\assets


...

INFO:tensorflow:Assets written to: primer_modelo\assets


1/1 [==============================] - 0s 135ms/step
INFO:tensorflow:Assets written to: C:\Users\el_he\AppData\Local\Temp\tmp6gr5ey4a\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\el_he\AppData\Local\Temp\tmp6gr5ey4a\model\data\model\assets
2023/10/02 23:50:33 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\el_he\anaconda3\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils."
2023/10/02 23:50:59 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\el_he\AppData\Local\Temp\tmpfp98av4y\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\el_he\AppData\Local\Temp\tmpfp98av4y\model\data\model\assets


In [18]:
!git status

On branch main

No commits yet

Changes to be committed:
  (use "git rm --cached <file>..." to unstage)
	new file:   primer_modelo/fingerprint.pb
	new file:   primer_modelo/keras_metadata.pb
	new file:   primer_modelo/saved_model.pb
	new file:   primer_modelo/variables/variables.data-00000-of-00001
	new file:   primer_modelo/variables/variables.index



In [19]:
!git add primer_modelo/saved_model.pb

In [20]:
!git commit -m "Subiendo la Red"

[main (root-commit) 47d8356] Subiendo la Red
 5 files changed, 16 insertions(+)
 create mode 100644 primer_modelo/fingerprint.pb
 create mode 100644 primer_modelo/keras_metadata.pb
 create mode 100644 primer_modelo/saved_model.pb
 create mode 100644 primer_modelo/variables/variables.data-00000-of-00001
 create mode 100644 primer_modelo/variables/variables.index


In [21]:
!git push

To https://dagshub.com/RZeusHerMun/PerrosGatos.git
 * [new branch]      main -> main
